In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import statsmodels.api as sm

In [3]:
### Aux Functions for reading two different .csv files ###

def read_hs1(yoozer):
    pd_df = pd.read_csv('../../Data/suggest-analysis-kristjan.csv') #'suggest-analysis-kristjan.csv')
    pdf = pd_df[:179]
    snda = pd_df['send.active'] == 1
    snd = pd_df['send'] == 0
    usr = pd_df['user'] == yoozer
    ddf = pd_df[(snda | snd) & usr]
    ddf = ddf.reset_index(drop=True)
    
    
    return ddf,pd_df

def read_hs1_gf(yoozer):
    pd_df = pd.read_csv('../../Data/suggest-kristjan.csv') #'suggest-analysis-kristjan.csv')
    pdf = pd_df[:179]
    snda = pd_df['send.active'] == 1
    snd = pd_df['send'] == 0
    usr = pd_df['user'] == yoozer
    ddf = pd_df[(snda | snd) & usr]
    ddf = ddf.reset_index(drop=True)

    return ddf,pd_df


def nan_equal(a,b):
    return ((a == b) | (np.isnan(a) & np.isnan(b))).all()


In [4]:
# Counts from HS v1

N = 48
T = 41
t = 5
nBaseline = 1+7
nInteract = 3 # Add 1 for bias term

In [5]:
### Read in Feature vector, Reward Vector, and Action Vector for each user ###
# Also Standardizes all features


max_yoozer = N+1

featVec = np.empty((max_yoozer,T*t, nBaseline))
featVec.fill(np.nan)

rewardVec = np.empty((max_yoozer,T*t))
rewardVec.fill(np.nan)

actionVec = np.empty((max_yoozer,T*t))
actionVec.fill(np.nan)

# Bias term

for yoozer in range(1,max_yoozer):
#     print("Processeing User: ", yoozer)

    ddf,pd_df = read_hs1(yoozer)
    #Make features
    #Center and scale
    decision_ind = ddf['decision.index.nogap']
    reward_h = ddf['jbsteps30.log']
    send_any = ddf['send']
    send_active = ddf['send.active']
    
    assert np.all(send_any == send_active)
    
    #total_sent = ddf['totalSent']
    # Study day index
    dazze = ddf['study.day.nogap']

    # Number of messages sent in last week
    day_ind  = (ddf['study.day.nogap'] - np.mean(pd_df['study.day.nogap']))/np.std(pd_df['study.day.nogap'])#Number sent in last whatever
    #Add feature for # of week period (hsteps v2) WATCH OUT FOR COLINEARITY WITH INTERCEPT
    
    # Ohter location indicator
    loc_ind = ddf['loc.is.other']
    
    # Std deviation of step count in last 7 days
    steps_sd = (ddf['window7.steps60.sd'] - np.mean(pd_df['window7.steps60.sd']))/np.std(pd_df['window7.steps60.sd'])
    
    # Step count in previous 30 minutes
    state = (ddf['jbsteps30pre.log'] - np.mean(pd_df['jbsteps30pre.log']))/np.std(pd_df['jbsteps30pre.log'])
    
    # Work location indicator
    #wrk_ind = ddf['location.category'], #loc_ind = ddf['location.category']
    wrk_ind = ddf['loc.is.work'] #compare to string “work”
    
    # Square root steps yesterday
    steps_yest = (ddf['steps.yesterday.sqrt'] - np.mean(pd_df['steps.yesterday.sqrt']))/np.std(pd_df['steps.yesterday.sqrt'])

    # Temperature, set -1024 as 0
    temp = (ddf['temperature'] - np.mean(pd_df['temperature']))/np.std(pd_df['temperature'])
    temp[ddf['temperature'] == -1024] = 0

    ddfgf,pd_dfgf = read_hs1_gf(yoozer)
    
    # ?? Unclear what this is
    # Reward: Step count in 30 minutes following decision point
    steps_gf = (np.log(ddfgf['gfsteps30pre'] + .5) - np.mean(np.log(pd_dfgf['gfsteps30pre'] + .5))/np.std(np.log(pd_dfgf['gfsteps30pre']+.5)))

    end_ind = day_ind.shape[0]

    
    # Set reward, action, and state
    rewardVec[yoozer,:end_ind] = reward_h.astype(float)
    actionVec[yoozer,:end_ind] = send_any.astype(float)
    
    featVec[yoozer,:end_ind,0].fill(1) # Only fill rows with observations
    featVec[yoozer,:end_ind,1] = day_ind #study.day.nogap
    featVec[yoozer,:end_ind,2] = loc_ind.astype(int) # loc.is.other
    featVec[yoozer,:end_ind,3] = steps_sd.astype(float) # window7.steps60.sd
    featVec[yoozer,:end_ind,4] = state # jbsteps30pre.log
    featVec[yoozer,:end_ind,5] = wrk_ind.astype(int) # loc.is.work
    featVec[yoozer,:end_ind,6] = steps_yest # steps.yesterday.sqrt
    featVec[yoozer,:end_ind,7] = temp.astype(float) # temperature
    
    # This was included but didn't make too much sense
#     featVec[yoozer,:end_ind,8] = steps_gf.astype(float)[:day_ind.shape[0]]

C:\Users\isaac\Anaconda2\envs\cs281\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (15,16,18,60,61,96,97,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
## TREAT DATA ##

# Drop 0th user, since users are 1-indexed
featVec = featVec[1:,:,:].copy()
rewardVec = rewardVec[1:,:].copy()
actionVec = actionVec[1:,:].copy()

# Reshape actionVec to have additional 1 dim
actionVec = actionVec.reshape(actionVec.shape[0],actionVec.shape[1],1)

# Mean impute featVec where there is an observation
featVec[~np.isnan(featVec[:,:,1])] = np.nan_to_num(featVec[~np.isnan(featVec[:,:,1])])

# Perform Regression on Pooled R ~ A*S' + S to create Residuals #

In [12]:
### Perform Regression on Pooled R ~ A*S' + S to create Residuals ###

# Actual Dimensions of A and S
a_dim = 1
s_dim = nBaseline

# Copy for notational ease
R = rewardVec.copy()
A = actionVec.copy()
S = featVec.copy()


# Fit OLS r ~ (a_t s'_t, s_t)^T \theta, where s' are interact terms and s are all baseline
resid_model = sm.OLS(endog = R.reshape(N*T*t), exog = np.concatenate([(A * S[:,:,1:1+nInteract]), S], 2).reshape((N*T*t,(nInteract + nBaseline))), missing = "drop")
resids = resid_model.fit().resid

# Parameters
Thetas_fit = resid_model.fit().params


## Fill Eta ##

# Copy shape and location of nans
Eta = rewardVec.copy()

curr_ind = 0
for n in range(N):
    
    old_ind = curr_ind
    curr_ind += R[n][~np.isnan(R[n])].shape[0]
    
    # Copy in indices
    Eta[n][:curr_ind-old_ind] = resids[old_ind:curr_ind]

# Code to Generate Simulated Users

In [9]:
def generate_new_users(Eta, A, S, N_new, T_new, T, t, users_to_sample = 10):
    '''
    Generates new random user
    
    Inputs:
        Eta: Residuals
        A: Actions
        S: States
        N_new: Int of number of new users to generate from sampling
        T_new: Int of number of days for each new user
    
    Returns:
        Eta_new: Matrix of Etas for sampled users
        A_new: Matrix of associated actions for sampled users
        S_new: Matrix of associated states for sampled users
        
    *Assumes shapes:
      Eta: (N, T * t) 
      A: (N, T * t, )
    '''
    
    # Obtain original dimensions of data from actions A
    N = A.shape[0]

    # Component dims of A and S 
    a_dim = A.shape[2]
    s_dim = S.shape[2]

    # Sample random users from original data
    sampled_user_indices = np.empty((N_new, users_to_sample)).astype(int)

    # Loop to reset sampling without replacement for each new user
    for i in range(N_new):
        sampled_user_indices[i] = np.random.choice(N, size = users_to_sample, replace = False)
    
    # Concatenate Eta, A, and S to ensure processing is uniform
    concat_data = np.concatenate([np.expand_dims(Eta,2),A,S], axis = 2)
    sampled_users_untreated = np.take(concat_data, sampled_user_indices, 0).reshape(N_new, users_to_sample * T*t, 1 + a_dim + s_dim)
    sampled_users = np.empty((N_new, T_new * t, 1 + a_dim + s_dim))
    
    for i in range(N_new):
        
        # Cast to DataFrame to dropna, reset index to shift datapoints up to be consecutive, takes T_new * t datapoints
        sampled_users[i] = pd.DataFrame(sampled_users_untreated[i]).apply(lambda col: col.dropna().reset_index().iloc[:,-1], axis = 0).as_matrix()[:T_new * t]
        
    # Sampled Generated residuals
    Eta_new = sampled_users[:,:,0].reshape(N_new, T_new, t)
    # Sampled actions
    A_new = sampled_users[:,:,1:(1+a_dim)].reshape(N_new, T_new, t, a_dim)
    # Sampled states
    S_new = sampled_users[:,:,(1+a_dim):].reshape(N_new, T_new, t, s_dim)
    
    return Eta_new, A_new, S_new


In [10]:
# Generate new Users
Eta_new, A_new, S_new = generate_new_users(Eta, A, S, 20, 90, T, t)


In [11]:
def reward_func(eta, a, s, Theta, nInteract = 3):
    '''Basic reward function, can edit for different generative models.
    Works for multidimensional eta, a, and s, so long as they are of same multidimension.
    Assumes s has first element as bias for the regression.'''
    return(eta + np.concatenate([a * np.take(s,range(1,1+nInteract),-1), s], a.ndim-1).dot(Theta))

print(reward_func(Eta_new, A_new, S_new, np.array(range(11))))
print(reward_func(Eta_new, A_new, S_new, Thetas_fit))

[[[-15.73433686 -18.28525317 -16.02196421   4.56949691   8.77649495]
  [ -1.09626857   4.06142394   1.71612255   7.6770708    5.03128181]
  [  6.29241288   8.7140227    7.92184904   1.83645888   2.05132275]
  ..., 
  [ 25.21814181  30.74522838  19.38541206  11.44059366  26.48328978]
  [ 21.8298586   14.58875834  16.69890944  32.61006702  27.09690726]
  [ 17.93618877  12.61667148  30.41349703  27.38341794   2.56620759]]

 [[-26.67556349 -17.73317739  -1.33857521 -14.24764243  -4.38909424]
  [  0.62335926  21.68276292  16.84246415   5.25067065  13.18966348]
  [ 15.73238684 -12.27717166   3.89798133   7.25996881  -5.56343015]
  ..., 
  [ 14.14581701  17.2369179   31.18702562  30.59133608  17.2381802 ]
  [ 13.19068685  10.53743576  27.10781956  14.90502437   0.92895807]
  [  7.49553799   7.53702882  19.69534325   5.22303868   0.66580707]]

 [[-16.81789062 -28.5355094  -16.7618992    0.03718345   1.8852322 ]
  [ 14.35911835  10.10058295  16.83172133  20.0627318    2.63535445]
  [ 14.4686649